<a href="https://colab.research.google.com/github/SaravanaPriyanD/Deep_Learning-Projects/blob/master/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print('cuda' if torch.cuda.is_available() else 'cpu')

cuda


In [0]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# Importing the dataset
movies = pd.read_csv('/content/drive/My Drive/Colab Files/Boltzmann_Machines/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/drive/My Drive/Colab Files/Boltzmann_Machines/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/drive/My Drive/Colab Files/Boltzmann_Machines/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [0]:
# Preparing the training set and the test set
training_set = pd.read_csv('/content/drive/My Drive/Colab Files/Boltzmann_Machines/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('/content/drive/My Drive/Colab Files/Boltzmann_Machines/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [0]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [9]:
nb_users
nb_movies

1682

In [0]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [13]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(1.7663)
epoch: 2 loss: tensor(1.0966)
epoch: 3 loss: tensor(1.0533)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0309)
epoch: 6 loss: tensor(1.0268)
epoch: 7 loss: tensor(1.0237)
epoch: 8 loss: tensor(1.0222)
epoch: 9 loss: tensor(1.0206)
epoch: 10 loss: tensor(1.0196)
epoch: 11 loss: tensor(1.0186)
epoch: 12 loss: tensor(1.0183)
epoch: 13 loss: tensor(1.0180)
epoch: 14 loss: tensor(1.0173)
epoch: 15 loss: tensor(1.0173)
epoch: 16 loss: tensor(1.0167)
epoch: 17 loss: tensor(1.0168)
epoch: 18 loss: tensor(1.0163)
epoch: 19 loss: tensor(1.0166)
epoch: 20 loss: tensor(1.0163)
epoch: 21 loss: tensor(1.0160)
epoch: 22 loss: tensor(1.0158)
epoch: 23 loss: tensor(1.0159)
epoch: 24 loss: tensor(1.0159)
epoch: 25 loss: tensor(1.0156)
epoch: 26 loss: tensor(1.0154)
epoch: 27 loss: tensor(1.0152)
epoch: 28 loss: tensor(1.0149)
epoch: 29 loss: tensor(1.0128)
epoch: 30 loss: tensor(1.0119)
epoch: 31 loss: tensor(1.0095)
epoch: 32 loss: tensor(1.0092)
epoch: 33 loss: t

In [16]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[(target == 0).unsqueeze(0)] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test loss: tensor(0.9498)
